Hi Ferdinand. I'm Svetlana and will review your project. I've added all my comments to new cells with different coloring. Please don't delete my comments.


The first time I see a mistake, I'll just point it out and let you find it and fix it yourself. In a real job, your boss will do the same, and I'm trying to prepare you to work as an Data Analyst. But if you can't handle this task yet, I'll give you a more accurate hint at the next check.

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
If you did something great I'm using green color for my comment
</div>

<div class="alert alert-warning" role="alert">
Reviewer's comment v. 1:
    
If I want to give you advice or think that something can be improved, then I'll use yellow. This is an optional recommendation.
</div>

<div class="alert alert-danger" role="alert">
Reviewer's comment v. 1:
    
If the topic requires some extra work so I can accept it then the color will be red
</div>

## Review

Your project looks pretty good! Glad to say that your project has been accepted. I'm impressed with the syntax, code formatting and detailed data analysis in your project. There are some comments for further improvement. They don't mean that you did anything wrong, but they're things we wanted to point out that can help you further develop your skills. Good luck on the next sprint!

## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

<div class="alert alert-warning" role="alert">
Reviewer's comment v. 1:
    
It’s great that you described the project, but it will be nice to add a table of contents with links.
    
Please see for details: https://www.markdowntutorial.com/
</div>

### Step 1. Open the data file and have a look at the general information. 

After reading in the data, I will look at the occurring values from each column using the value_counts() function. This way I will find missing values, wrong values and duplicates.

In [1]:
import pandas as pd 
import numpy as np

    
data = pd.read_csv("/datasets/credit_scoring_eng.csv")
data.info()
data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
Hint: Please note that you can use data.head(N) to see N first rows of the data dataframe and data.tail(N) to see last N rows of the data dataframe.
</div>

In [2]:
data["children"].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [3]:
#as you see: all rows with missing values in the column "days_employed" also have missing values in the column "total_income"
#but most of them still have values in the column "income_type" like "employee" or "business", so they should have an income
#for civil servants it would be logical, if they have no income
print("count of missing values in total_income and days employed:")
print("--------------")
print(data.loc[(data["days_employed"].isna()) & (data["total_income"].isna())].count())
print()

#many unrealitic values: people work about 50 years --> 50*365 = 18.250 [days]
#mean in this coulmn: 66915 [days]... that is more than twice the life expectancy of a human being
av_daysEmpl = data["days_employed"].mean()
print("mean of the column days_employed:")
print("--------------")
print(av_daysEmpl)
print()
print("income_type of the people with missing values in days_employed")
print("--------------")
print()

#poeple, who are employed for more than 18.250 days (50 years) must be at least older than 65
#as you see, the majority of these people are younger than 65
#so i guess, that this table cant be processed without changing it completely
print("Ages of the people, who are employed for more than 18.250 [days] (= 50 years!!!) due to the data:")
print("--------------")
print(data.loc[data["days_employed"] > 18250]["dob_years"].value_counts().sort_index())
print()
#2745 people are younger than 65 but worked for more than 50 years due to this data
sum = data.loc[(data["days_employed"] > 18250) & (data["dob_years"] < 65)]["dob_years"].value_counts().sum()
print("Sum of the people, who are employed for more than 18.250 [days] (= 50 years!!!) beyond the age of 66")
print("--------------")
print(sum)
print()
print(data.loc[(data["days_employed"].isna()) & (data["total_income"].isna())]["income_type"].value_counts())
["income_type"]

count of missing values in total_income and days employed:
--------------
children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

mean of the column days_employed:
--------------
63046.49766147338

income_type of the people with missing values in days_employed
--------------

Ages of the people, who are employed for more than 18.250 [days] (= 50 years!!!) due to the data:
--------------
0      17
22      1
26      2
27      3
28      1
31      1
32      3
33      2
34      3
35      1
36      3
37      5
38      8
39      4
40      7
41      6
42      9
43      9
44     10
45     11
46     13
47     13
48     20
49     30
50     61
51     73
52     95
53    105
54    145
55    162
56    184
57    212
58    208
59    254
60    243
61

['income_type']

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
Well done that you printed the information about data.
    
Please note that you can use .format to output only required number of decimal places (https://pyformat.info/).
</div>

In [4]:
data["dob_years"].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [5]:
data["education"].value_counts().sort_index()

BACHELOR'S DEGREE        274
Bachelor's Degree        268
GRADUATE DEGREE            1
Graduate Degree            1
PRIMARY EDUCATION         17
Primary Education         15
SECONDARY EDUCATION      772
SOME COLLEGE              29
Secondary Education      711
Some College              47
bachelor's degree       4718
graduate degree            4
primary education        250
secondary education    13750
some college             668
Name: education, dtype: int64

In [6]:
data["family_status"].value_counts().sort_index()

civil partnership     4177
divorced              1195
married              12380
unmarried             2813
widow / widower        960
Name: family_status, dtype: int64

In [7]:
data["gender"].value_counts().sort_index()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [8]:
data["income_type"].value_counts().sort_index()

business                        5085
civil servant                   1459
employee                       11119
entrepreneur                       2
paternity / maternity leave        1
retiree                         3856
student                            1
unemployed                         2
Name: income_type, dtype: int64

In [9]:
data["debt"].value_counts().sort_index()

0    19784
1     1741
Name: debt, dtype: int64

In [10]:
data["total_income"].sort_values()

14585    3306.762
13006    3392.845
16174    3418.824
1598     3471.216
14276    3503.298
           ...   
21489         NaN
21495         NaN
21497         NaN
21502         NaN
21510         NaN
Name: total_income, Length: 21525, dtype: float64

In [11]:
data["purpose"].value_counts().sort_index()

building a property                         620
building a real estate                      626
buy commercial real estate                  664
buy real estate                             624
buy residential real estate                 607
buying a second-hand car                    479
buying my own car                           505
buying property for renting out             653
car                                         495
car purchase                                462
cars                                        478
construction of own property                635
education                                   447
getting an education                        443
getting higher education                    426
going to university                         496
having a wedding                            777
housing                                     647
housing renovation                          612
housing transactions                        653
profile education                       

### Conclusion

1. Datatypes: 
              -missing data in columns "days_employed" and "total_income" (both have 19351 entries)
              -datatype of "days_employed" is float... int would make more sense
2. Columns:
    2.1 children: 
                -76x 20 children seems unrealistic (6-19 children never appears)
                -negative value: -1 children
    2.2 days_employed:
                -theses data have to many unrealistic values to process them without changing them completely:
                    -people work about 50 years --> 50*365 = 18.250 [days], but the mean value is 66915 [days]
                    that is more than twice the life expectancy of a human being
                    -2881 people in this data are younger than 65 but worked for more than 50 years (18.250 days)
                -negative values
                -float datatype
                -missing values (people who also have missing values in column "total_income")   
                    
    2.3 dob_years:
                -0 must be a mistake
    2.4 education:
                -evry variable appears 3 times
    2.5 family_status:
                -seems okay
    2.6 gender:
                -seems okay
                -women are included twice as much in the data
    2.7 income_type:
                -seems okay
    2.8 debt:
                -seems okay
    2.9 total_income:
                -missing values (people who also have missing values in column "days_employed")
    2.10 purpose:
                -all variables appear more than one time

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
You have made a really detailed analysis of the problems with data :)
    
Well done that you highlighted problems with "days_employed" column.
</div>

## 2. Preprocessing

steps:
    -replace negative values (children, days_employed)
    -eliminate similar designations (education, purpose)
    -eliminate duplicates
    -replace missing values by mean (days_employed, total_income)
    -replace wrong values by medians (children, dob_years)
    -categorizing the data to answer the questions (children, family_status, total_income, purpose)

### Replace negative values (children, days_employed)

In [12]:
print(data["children"].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [13]:
data["children"] = data["children"].abs()
print(data["children"].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


In [14]:
print(data.loc[data["days_employed"].notna()]["days_employed"].sort_values(ascending= False))

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
             ...      
16825    -16119.687737
17838    -16264.699501
7329     -16593.472817
4299     -17615.563266
16335    -18388.949901
Name: days_employed, Length: 19351, dtype: float64


In [15]:
data["days_employed"] = data["days_employed"].abs()
print(data.loc[data["days_employed"].notna()]["days_employed"].sort_values(ascending= False))

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
             ...      
2127         34.701045
9683         33.520665
6157         30.195337
8336         24.240695
17437        24.141633
Name: days_employed, Length: 19351, dtype: float64


### Eliminate similar designations (education, purpose)

The duplicates most likely occur because either different people were involved in the data collection. It is also possible that the data came from different sources, with each source having its own name for the variables.

In [16]:
print("education values:")
print("--------------")
data["education"].value_counts()
print(data["education"].value_counts())

education values:
--------------
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64


In [17]:
data.loc[(data["education"] == "BACHELOR'S DEGREE") | (data["education"] == "Bachelor's Degree") | (data["education"] == "bachelor's degree"), "education"] = "bachelor's degree"
data.loc[(data["education"] == "SECONDARY EDUCATION") | (data["education"] == "Secondary Education"), "education"] = "secondary education"
data.loc[(data["education"] == "SOME COLLEGE") | (data["education"] == "Some College"), "education"] = "some college"
data.loc[(data["education"] == "PRIMARY EDUCATION") | (data["education"] == "Primary Education"), "education"] = "primary education"
data.loc[(data["education"] == "GRADUATE DEGREE") | (data["education"] == "Graduate Degree"), "education"] = "graduate degree"



print("education values:")
print("--------------")
data["education"].value_counts()
print(data["education"].value_counts())


education values:
--------------
secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
Great education values are prepared for the further analysis.
</div>

In [18]:
print(data["purpose"].value_counts())


wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
housing transactions                        653
buying property for renting out             653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
purchase of my own house                    620
building a property                         620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

In this case I will use a loop for the list "test_duplicates_list", which contains words that appear more than once in the printed list above. The Loop will go through the data and replaces the duplicates with a more general description. This Function takes a lot of time because it consists of 3 loops.

In [19]:
from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')

purpose_list = data["purpose"].to_list()
test_duplicates_list = ["house", "car", "education", "estate", "wedding", "property"]

for test_word in test_duplicates_list:

    for purpose in purpose_list:
        duplicate_purpose = False

        for word in purpose.split(' '):
            stemmed_word = english_stemmer.stem(word) 
            if stemmed_word == english_stemmer.stem(test_word):
                duplicate_purpose = True
        if duplicate_purpose == True:
            data.loc[data["purpose"] == purpose, "purpose"] = test_word
              
        


<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
It's great that you have used nltk. 
    
Maybe this link will be interesting for you: https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/ 
</div>

In [20]:
print(data["purpose"].value_counts())   

estate                 4478
car                    4315
house                  3820
education              3526
property               2542
wedding                2348
going to university     496
Name: purpose, dtype: int64


In the list are still values that are very similar in meaning. Therefore, I will modify them manually.

In [21]:
data.loc[(data["purpose"] == "estate") | (data["purpose"] == "property"), "purpose"] = "house"
data.loc[data["purpose"] == "going to university", "purpose"] = "education"

print(data["purpose"].value_counts()) 

house        10840
car           4315
education     4022
wedding       2348
Name: purpose, dtype: int64


### Eliminate duplicates

In [22]:
#there are still 21525 rows
print(len(data))

21525


In [23]:
#I just use the drop_duplicates() function:

data = data.drop_duplicates()
print(len(data))

21120


In [24]:
print("{} rows were removed!".format(21525-21115))

410 rows were removed!


<div class="alert alert-warning" role="alert">
Reviewer's comment v. 1:
    
Could ypu please calculate the percentage of duplicates in our data before you dropped it?
</div>

### Replace missing values by mean (total_income)

As we have already seen, the "total_income" values are missing for exactly those persons who also have missing values in the "days_employed" column. These are very likely related.

In [25]:
#calculating the number of missing values
print(data.loc[data["total_income"].isna()].count())


children            1769
days_employed          0
dob_years           1769
education           1769
education_id        1769
family_status       1769
family_status_id    1769
gender              1769
income_type         1769
debt                1769
total_income           0
purpose             1769
dtype: int64


Before replacing the missing values in "total_income", I will test whether there are large differences in average incomes as a function of educational attainment. If this is the case, I will calculate a separate income average for each educational attainment and replace the missing values in the column "total_income" with these.

In [26]:
# calculating the average of "total_income" in dependency of the education --> there is a dependency
print("education values:")
print("--------------")
data["education"].value_counts()
print(data["education"].value_counts())
totInc_bacDeg_av = data.loc[data["education"] == "bachelor's degree"]["total_income"].mean()
totInc_secEdu_av = data.loc[(data["education"] == "secondary education")]["total_income"].mean()
totInc__col_av = data.loc[(data["education"] == "some college")]["total_income"].mean()
totInc_primEdu_av = data.loc[(data["education"] == "primary education")]["total_income"].mean()
totInc_gradDeg_av = data.loc[(data["education"] == "graduate degree")]["total_income"].mean()

print()
print("total_income means in dependency of the education")
print("--------------")
print("Bachelor Degree: {:.2f}".format(totInc_bacDeg_av))
print("Secondary Education: {:.2f}".format(totInc_secEdu_av))
print("Some College: {:.2f}".format(totInc__col_av))
print("Primary Education: {:.2f}".format(totInc_primEdu_av))
print("Graduate Degree: {:.2f}".format(totInc_gradDeg_av))


education values:
--------------
secondary education    14876
bachelor's degree       5212
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

total_income means in dependency of the education
--------------
Bachelor Degree: 33142.80
Secondary Education: 24594.50
Some College: 29045.44
Primary Education: 21144.88
Graduate Degree: 27960.02


In [27]:
#replace the missing values by mean
data.loc[(data["education"] == "bachelor's degree") & (data["total_income"].isna()), "total_income"] = totInc_bacDeg_av

data.loc[(data["education"] == "secondary education") & (data["total_income"].isna()), "total_income"] = totInc_secEdu_av
data.loc[(data["education"] == "some college") & (data["total_income"].isna()), "total_income"] = totInc__col_av
data.loc[(data["education"] == "primary education") & (data["total_income"].isna()), "total_income"] = totInc_primEdu_av
data.loc[(data["education"] == "graduate degree") & (data["total_income"].isna()), "total_income"] = totInc_gradDeg_av
data.loc[(data["education"] == "bachelor's degree") & (data["total_income"].isna()), "total_income"] = totInc_bacDeg_av


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<div class="alert alert-warning" role="alert">
Reviewer's comment v. 1:
    
Could you please explain why you filled a missing values by mean?
    
Great that you filled a missing values by mean. However please note that it's also possible to change a missing values to median, mode, 0 or other values. It depends on our data.
    
Additional links: https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html, https://thispointer.com/python-pandas-count-number-of-nan-or-missing-values-in-dataframe-also-row-column-wise/,
https://www.geeksforgeeks.org/working-with-missing-data-in-pandas/
</div>

In [28]:
#calculating the number of missing values again
print(data.loc[data["total_income"].isna()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Replace wrong values by median (children, dob_years)

In the column "children" the value 20 appears 76 times. This must be a mistake. I will replace these values by the median of the column.

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
You have made a really detailed analysis of the problems with data :)
    
Well done that you highlighted problems with "days_employed" column.
</div>

In [29]:
data["children"].value_counts()

0     13829
1      4798
2      2039
3       329
20       76
4        40
5         9
Name: children, dtype: int64

In [30]:
median_children = data.loc[data["children"] < 20]["children"].median()
data.loc[data["children"] > 19, "children"] = median_children


In [31]:
data["children"].value_counts()

0.0    13905
1.0     4798
2.0     2039
3.0      329
4.0       40
5.0        9
Name: children, dtype: int64

In the column "dob_years" the value 0 appears 101 times. This also must be a mistake. I will also replace these values by the median of the column.

<div class="alert alert-warning" role="alert">
Reviewer's comment v. 1:
    
Could you please explain why you filled a missing values in the column "dob_years" by median?
</div>

In [32]:
print(data["dob_years"].value_counts().sort_index())

0     100
19     14
20     51
21    110
22    183
23    244
24    264
25    357
26    403
27    488
28    498
29    539
30    530
31    555
32    506
33    572
34    599
35    605
36    547
37    534
38    592
39    563
40    600
41    599
42    583
43    504
44    538
45    488
46    468
47    460
48    529
49    494
50    504
51    439
52    473
53    450
54    464
55    435
56    471
57    445
58    442
59    438
60    363
61    346
62    340
63    262
64    250
65    189
66    177
67    164
68     98
69     85
70     65
71     55
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [33]:
median_dob_years = data.loc[data["dob_years"] > 0]["dob_years"].median()


data.loc[data["dob_years"] == 0, "dob_years"] = median_dob_years

In [34]:
print(data["dob_years"].value_counts().sort_index())

19.0     14
20.0     51
21.0    110
22.0    183
23.0    244
24.0    264
25.0    357
26.0    403
27.0    488
28.0    498
29.0    539
30.0    530
31.0    555
32.0    506
33.0    572
34.0    599
35.0    605
36.0    547
37.0    534
38.0    592
39.0    563
40.0    600
41.0    599
42.0    683
43.0    504
44.0    538
45.0    488
46.0    468
47.0    460
48.0    529
49.0    494
50.0    504
51.0    439
52.0    473
53.0    450
54.0    464
55.0    435
56.0    471
57.0    445
58.0    442
59.0    438
60.0    363
61.0    346
62.0    340
63.0    262
64.0    250
65.0    189
66.0    177
67.0    164
68.0     98
69.0     85
70.0     65
71.0     55
72.0     33
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64


### Categorizing Data (children, family_status, total_income, purpose)

To answer one of the questions of the project, I will create the column "at_least_one_child" with the possible values "True" and "False", so evrybody who has at least one child gets the value "True" and the rest the value "False".

In [35]:
print(data["children"].value_counts())

0.0    13905
1.0     4798
2.0     2039
3.0      329
4.0       40
5.0        9
Name: children, dtype: int64


In [36]:
def at_least_one_child(children):
    if children > 0:
        return "True"
    else:
        return "False"
    
    
data["has_child(ren)"] = data["children"].apply(at_least_one_child)

print(data.head())
    

   children  days_employed  dob_years            education  education_id  \
0       1.0    8437.673028       42.0    bachelor's degree             0   
1       1.0    4024.803754       36.0  secondary education             1   
2       0.0    5623.422610       33.0  secondary education             1   
3       3.0    4124.747207       32.0  secondary education             1   
4       0.0  340266.072047       53.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

     purpose has_child(ren)  
0      house           Tru

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<div class="alert alert-warning" role="alert">
Reviewer's comment v. 1:
    
For more detailed analysis it will be better to divide into more than one cathegory (0, 1, 2, 3 and more children).
</div>

To answer one of the questions of the project, I will put "unmarried", "divorced" and "widow / widower" in one category because it means there was at least one marriage. The weakness of this model is that you don't know for sure if the late loan repaying was during the same time of the marriage, but it's possible.

In [37]:
print(data["family_status"].value_counts())

married              12076
civil partnership     4124
unmarried             2784
divorced              1193
widow / widower        943
Name: family_status, dtype: int64


In [38]:
def married_or_onced_married(family_status):
    if (family_status == "married") | (family_status == "divorced") | (family_status == "widow / widower"):
        return "True"
    else:
        return "False"
    
data["married_or_onced_married"] = data["family_status"].apply(married_or_onced_married)
print(data.head())

   children  days_employed  dob_years            education  education_id  \
0       1.0    8437.673028       42.0    bachelor's degree             0   
1       1.0    4024.803754       36.0  secondary education             1   
2       0.0    5623.422610       33.0  secondary education             1   
3       3.0    4124.747207       32.0  secondary education             1   
4       0.0  340266.072047       53.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

     purpose has_child(ren) married_or_onced_married  
0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


To answer one of the questions of the project, I will add the coulmn "income_level" with the possible values "low", "middle", "high" and "very high" in dependency of the "total_income" values. Because there is no information about the country were this data come from, it's not easy to get the right levels, that's why i experminted a bit and came to the following levels:

In [39]:
very_low_income = data.loc[data["total_income"] < 12000]["total_income"].count()
low_income = data.loc[(12000 <= data["total_income"]) & (data["total_income"] < 22000)]["total_income"].count()
middle_income =data.loc[(22000 <= data["total_income"]) & (data["total_income"] < 32000)]["total_income"].count()
high_income = data.loc[(32000 <= data["total_income"]) & (data["total_income"] < 42000)]["total_income"].count()
very_high_income = data.loc[(42000 <= data["total_income"])]["total_income"].count()

print("people with very low income: {}".format(very_low_income))
print("people with low income: {}".format(low_income))
print("people with very middle income: {}".format(middle_income))
print("people with high income: {}".format(high_income))
print("people with very high income: {}".format(very_high_income))



people with very low income: 1865
people with low income: 6970
people with very middle income: 6723
people with high income: 3203
people with very high income: 2359


<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
Please note that you can avoid warnings (if necessary) by using the code below.
</div>

In [46]:
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")

In [47]:
def income_level(total_income):
    
    if total_income < 12000:
        return "very low"
    elif 12000 <= total_income < 22000:
        return "low"
    elif 22000 <= total_income < 32000:
        return "middle"
    elif 32000 <= total_income < 42000:
        return "high"
    elif 42000 <= total_income:
        return "very high"

data["income_level"] = data["total_income"].apply(income_level)

print(data.head(10))

   children  days_employed  dob_years            education  education_id  \
0       1.0    8437.673028       42.0    bachelor's degree             0   
1       1.0    4024.803754       36.0  secondary education             1   
2       0.0    5623.422610       33.0  secondary education             1   
3       3.0    4124.747207       32.0  secondary education             1   
4       0.0  340266.072047       53.0  secondary education             1   
5       0.0     926.185831       27.0    bachelor's degree             0   
6       0.0    2879.202052       43.0    bachelor's degree             0   
7       0.0     152.779569       50.0  secondary education             1   
8       2.0    6929.865299       35.0    bachelor's degree             0   
9       0.0    2188.756445       41.0  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

The "purpose"- column is already categorized because i summarized many values already as i removed duplicates.

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
You categorized the data correctly.
</div>

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

To answer this, i devide the ammount of people, who have children and repaid a loan lately by the total number of people who have children ("children") and save it to the variable "children_debt". Analogous to this, i will create the variable "no_children_debt" and compare these two.

In [41]:
children = len(data.loc[data["has_child(ren)"] == "True"])


no_children = len(data.loc[data["has_child(ren)"] == "False"])



children_debt = len(data.loc[(data["has_child(ren)"] == "True") & (data["debt"] == 1)]) / children

no_children_debt = len(data.loc[(data["has_child(ren)"] == "False") & (data["debt"] == 1)]) / no_children

print("The proportion of people, who repaid a loan lately and have children out of the people with children: {:.2%}".format(children_debt))
print("The proportion of people, who repaid a loan lately and have no children out of the people without children: {:.2%}".format(no_children_debt))

print("Someone who has children is {:.2%} more likely to be in credit default than someone who does not have children.".format(children_debt/no_children_debt-1))


The proportion of people, who repaid a loan lately and have children out of the people with children: 9.29%
The proportion of people, who repaid a loan lately and have no children out of the people without children: 7.69%
Someone who has children is 20.79% more likely to be in credit default than someone who does not have children.


<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
An excellent analysis. However please note that you can also use a correlation function. The correlation function shows only linear dependecy between variables. Maybe this link will be interesting for you: https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/.
</div>

- Is there a relation between marital status and repaying a loan on time?

To answer this, i devide the ammount of people, who are / were married and repaid a loan lately by the total number of people who are / were married and save it to the variable "marriage_debt". Analogous to this, i will create the variable "not_married_debt" and compare these two

In [42]:
marriage = len(data.loc[data["married_or_onced_married"] == "True"])


no_marriage = len(data.loc[data["married_or_onced_married"] == "False"])



marriage_debt = len(data.loc[(data["married_or_onced_married"] == "True") & (data["debt"] == 1)]) / marriage

no_marriage_debt = len(data.loc[(data["married_or_onced_married"] == "False") & (data["debt"] == 1)]) / no_marriage

print("The proportion of people, who repaid a loan lately and  are / were married out of the people, who are / were married: {:.2%}".format(marriage_debt))
print("The proportion of people, who repaid a loan lately and  were never married out of the people, who were never married: {:.2%}".format(no_marriage_debt))
print("Someone who has never been married is {:.2%} more likely to have entered into a loan default than someone who has been or is married.".format(no_marriage_debt/marriage_debt-1))


The proportion of people, who repaid a loan lately and  are / were married out of the people, who are / were married: 7.58%
The proportion of people, who repaid a loan lately and  were never married out of the people, who were never married: 9.58%
Someone who has never been married is 26.46% more likely to have entered into a loan default than someone who has been or is married.


<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
Yes, it's interesting result.
</div>

- Is there a relation between income level and repaying a loan on time?

To answer this, I divide the number of people who have a very low income and once fell into credit default by the total number of people who have a very low income and store this in the variable "very_low_income_debt"
analogously, I create the variables "low_income_debt", "middle_income_debt", "high_income_debt" and "very_high_income_debt" and compare them with each other.

In [43]:
very_low_income = data.loc[data["total_income"] < 12000]["total_income"].count()
low_income = data.loc[(12000 <= data["total_income"]) & (data["total_income"] < 22000)]["total_income"].count()
middle_income =data.loc[(22000 <= data["total_income"]) & (data["total_income"] < 32000)]["total_income"].count()
high_income = data.loc[(32000 <= data["total_income"]) & (data["total_income"] < 42000)]["total_income"].count()
very_high_income = data.loc[(42000 <= data["total_income"])]["total_income"].count()

very_low_income_debt = len(data.loc[(data["income_level"] == "very low") & (data["debt"] == 1)]) / very_low_income
low_income_debt = len(data.loc[(data["income_level"] == "low") & (data["debt"] == 1)]) / low_income
middle_income_debt = len(data.loc[(data["income_level"] == "middle") & (data["debt"] == 1)]) / middle_income
high_income_debt = len(data.loc[(data["income_level"] == "high") & (data["debt"] == 1)]) / high_income
very_high_income_debt = len(data.loc[(data["income_level"] == "very high") & (data["debt"] == 1)]) / very_high_income


print("people with very low income: {}".format(very_low_income))
print("people with low income: {}".format(low_income))
print("people with middle income: {}".format(middle_income))
print("people with high income: {}".format(high_income))
print("people with very high income: {}".format(very_high_income))
print()
print("The percentage of people who have very low incomes and were once in credit default: {:.2%}".format(very_low_income_debt))
print("The percentage of people who have low incomes and were once in credit default: {:.2%}".format(low_income_debt))
print("The percentage of people who have middle incomes and were once in credit default: {:.2%}".format(middle_income_debt))
print("The percentage of people who have high incomes and were once in credit default: {:.2%}".format(high_income_debt))
print("The percentage of people who have very high incomes and were once in credit default: {:.2%}".format(very_high_income_debt))
print()
print("Compared to those with middle incomes, those with...")
print("very low incomes are {:.2%} less likely to be in credit default.".format(very_low_income_debt / middle_income_debt-1))
print("low incomes are {:.2%} less likely to be in credit default.".format(low_income_debt / middle_income_debt-1))
print("high incomes are {:.2%} less likely to be in credit default.".format(high_income_debt / middle_income_debt-1))
print("very high incomes are {:.2%} less likely to be in credit default.".format(very_high_income_debt / middle_income_debt-1))



people with very low income: 1865
people with low income: 6970
people with middle income: 6723
people with high income: 3203
people with very high income: 2359

The percentage of people who have very low incomes and were once in credit default: 7.29%
The percentage of people who have low incomes and were once in credit default: 8.51%
The percentage of people who have middle incomes and were once in credit default: 9.31%
The percentage of people who have high incomes and were once in credit default: 6.90%
The percentage of people who have very high incomes and were once in credit default: 6.91%

Compared to those with middle incomes, those with...
very low incomes are -21.68% less likely to be in credit default.
low incomes are -8.63% less likely to be in credit default.
high incomes are -25.90% less likely to be in credit default.
very high incomes are -25.79% less likely to be in credit default.


This means that middle-income people are the most likely to default on their loans, followed by low-income people, very low-income people, high-income people and very high-income people. As we can see, there is no clear pattern.

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
Yes, it's interesting result that middle-income people are the most likely to default on their loans. Could you please try to explain it?
</div>

- How do different loan purposes affect on-time repayment of the loan?

In [44]:
print(data["purpose"].value_counts())

house        10578
car           4272
education     3964
wedding       2306
Name: purpose, dtype: int64


To answer this, I divide the number of people who have specified "house" as the purpose and have defaulted once by the total number of people who have specified "marriage" as the purpose and store the result in the variable "wedding_debt".
Similarly, I create the variables "car_debt", "education_debt" and "wedding" and compare them with each other.

In [45]:
house = len(data.loc[data["purpose"] == "house"])
car = len(data.loc[data["purpose"] == "car"])
education = len(data.loc[data["purpose"] == "education"])
wedding = len(data.loc[data["purpose"] == "wedding"])

house_debt = len(data.loc[(data["purpose"] == "house") & (data["debt"] == 1)]) / house
car_debt = len(data.loc[(data["purpose"] == "car") & (data["debt"] == 1)]) / car
education_debt = len(data.loc[(data["purpose"] == "education") & (data["debt"] == 1)]) / education
wedding_debt = len(data.loc[(data["purpose"] == "wedding") & (data["debt"] == 1)]) / wedding


print("The percentage of people who indicated 'house' as their purpose and were once in credit default: {:.2%}".format(house_debt))
print("The percentage of people who indicated 'car' as their purpose and were once in credit default: {:.2%}".format(car_debt))
print("The percentage of people who indicated 'education' as their purpose and were once in credit default: {:.2%}".format(education_debt))
print("The percentage of people who indicated 'wedding' as their purpose and were once in credit default: {:.2%}".format(wedding_debt))

print()

print("Compared to those who indicated 'house' as their purpose, those who indicated...")
print("'car' as their purpose were {:.2%} more likely to be in loan default.".format(car_debt / house_debt - 1))
print("'education' as their purpose were {:.2%} more likely to be in loan default.".format(education_debt / house_debt - 1))      
print("'wedding' as their purpose were {:.2%} more likely to be in loan default.".format(wedding_debt / house_debt - 1))


The percentage of people who indicated 'house' as their purpose and were once in credit default: 7.38%
The percentage of people who indicated 'car' as their purpose and were once in credit default: 9.41%
The percentage of people who indicated 'education' as their purpose and were once in credit default: 9.33%
The percentage of people who indicated 'wedding' as their purpose and were once in credit default: 8.07%

Compared to those who indicated 'house' as their purpose, those who indicated...
'car' as their purpose were 27.45% more likely to be in loan default.
'education' as their purpose were 26.42% more likely to be in loan default.
'wedding' as their purpose were 9.25% more likely to be in loan default.


People who listed "car" as their purpose were most likely to default on their loans, followed by "education," "wedding" and "house".

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
It's great that you analysed data in details :)
</div>

### Step 4. General conclusion

In this dataset, there were missing values, duplicates and wrong values like negative or unrealistic values. Especially the column "days_employed" can only be evaluated very difficult, because of the unrealistic values already mentioned above.

The questions of this task were whether there is a relationship between on-time repayment of a loan and certain variables.
Summarizing you can say, that these properties were most closely related to credit default:

having childs, not being married, middle-income level and car as purpose for the loan.

<div class="alert alert-success" role="alert">
Reviewer's comment v. 1:
    
An excellent conclusion which highlights all the main results of the project. Definitely this information will be useful for a bank’s loan division.
</div>

### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [ ]  replaced the real data type with an integer;
- [ ]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.